In [1]:
import requests
import pandas as pd

def get_btc_option_ticker():
    url = "https://eapi.binance.com/eapi/v1/ticker"
    response = requests.get(url)
    
    print(f"[STATUS CODE] {response.status_code}")
    
    if response.status_code != 200:
        print("[ERROR] 응답 실패:", response.text[:300])
        return None
    
    data = response.json()
    df = pd.DataFrame(data)
    return df[df['symbol'].str.startswith("BTC")]

btc_df = get_btc_option_ticker()
print(btc_df.head())


[STATUS CODE] 200
                symbol priceChange priceChangePercent lastPrice lastQty  \
1  BTC-251226-120000-P           0                  0         0       0   
2  BTC-250725-130000-C        1090             0.3733      4010       0   
4   BTC-250523-92000-C           0                  0     10830     2.6   
5  BTC-250725-116000-C        2675             0.6115      7050     1.1   
6  BTC-250606-116000-P       -5250            -0.3796      8580     1.2   

    open   high    low volume   amount bidPrice askPrice       openTime  \
1      0      0      0      0        0    19445        0              0   
2   2920   4010   2615   2.15  6583.75     3850     3920  1747809026882   
4  10830  10830  10830      0        0        0        0              0   
5   4375   7050   4375   1.15   8107.5     7800     7910              0   
6  13830  13830   8530   4.56  38963.6     6865     6965              0   

       closeTime  firstTradeId  tradeCount strikePrice   exercisePrice  
1      

In [2]:
import requests

def get_deribit_option_instruments(currency="BTC"):
    url = f"https://www.deribit.com/api/v2/public/get_instruments?currency={currency}&kind=option"
    r = requests.get(url)
    return r.json()['result']

In [3]:
get_deribit_option_instruments()

[{'price_index': 'btc_usd',
  'rfq': False,
  'kind': 'option',
  'min_trade_amount': 0.1,
  'instrument_name': 'BTC-22MAY25-92000-C',
  'maker_commission': 0.0003,
  'taker_commission': 0.0003,
  'instrument_type': 'reversed',
  'expiration_timestamp': 1747900800000,
  'creation_timestamp': 1747641609000,
  'is_active': True,
  'tick_size': 0.0001,
  'contract_size': 1.0,
  'strike': 92000.0,
  'instrument_id': 465833,
  'settlement_period': 'day',
  'option_type': 'call',
  'block_trade_commission': 0.0003,
  'block_trade_min_trade_amount': 25,
  'block_trade_tick_size': 0.0001,
  'settlement_currency': 'BTC',
  'base_currency': 'BTC',
  'counter_currency': 'USD',
  'quote_currency': 'BTC',
  'tick_size_steps': [{'tick_size': 0.0005, 'above_price': 0.005}]},
 {'price_index': 'btc_usd',
  'rfq': False,
  'kind': 'option',
  'min_trade_amount': 0.1,
  'instrument_name': 'BTC-22MAY25-92000-P',
  'maker_commission': 0.0003,
  'taker_commission': 0.0003,
  'instrument_type': 'reversed',
 

In [4]:
import requests
import pandas as pd
from datetime import datetime

def get_option_candles(instrument_name, start_time_ms, end_time_ms, resolution='60'):
    """
    Deribit 옵션 캔들 데이터 가져오기 (모든 필드 포함)
    resolution: '1', '5', '15', '30', '60', '120', '240', 'D'
    """
    url = "https://www.deribit.com/api/v2/public/get_tradingview_chart_data"
    params = {
        "instrument_name": instrument_name,
        "start_timestamp": start_time_ms,
        "end_timestamp": end_time_ms,
        "resolution": resolution
    }
    r = requests.get(url, params=params)
    result = r.json().get("result", {})
    df = pd.DataFrame(result)

    if not df.empty:
        df['timestamp'] = pd.to_datetime(df['ticks'], unit='ms')
        df['instrument'] = instrument_name  # 어떤 옵션인지 표시
        df = df[[
            'timestamp', 'open', 'high', 'low', 'close', 'volume', 'cost', 'instrument'
        ]]
    return df

# 예시 사용
instrument = "BTC-28JUN24-30000-C"  # 원하는 옵션 심볼
end = int(datetime.now().timestamp() * 1000)
start = end - 30 * 24 * 60 * 60 * 1000  # 과거 30일치

df = get_option_candles(instrument, start, end, resolution='60')
print(df[['timestamp', 'open', 'high', 'low', 'close', 'volume']].head())


            timestamp    open    high     low   close  volume
0 2025-04-22 05:00:00  0.5095  0.5095  0.5095  0.5095       0
1 2025-04-22 06:00:00  0.5095  0.5095  0.5095  0.5095       0
2 2025-04-22 07:00:00  0.5095  0.5095  0.5095  0.5095       0
3 2025-04-22 08:00:00  0.5095  0.5095  0.5095  0.5095       0
4 2025-04-22 09:00:00  0.5095  0.5095  0.5095  0.5095       0


In [5]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# 1. 모든 BTC 옵션 심볼 가져오기
def get_deribit_option_instruments(currency="BTC"):
    url = f"https://www.deribit.com/api/v2/public/get_instruments?currency={currency}&kind=option"
    r = requests.get(url)
    return r.json()['result']

# 2. 특정 옵션에 대한 OHLC 캔들 데이터 가져오기 (시간 단위)
def get_option_candles(instrument_name, start_time_ms, end_time_ms, resolution='60'):
    url = "https://www.deribit.com/api/v2/public/get_tradingview_chart_data"
    params = {
        "instrument_name": instrument_name,
        "start_timestamp": start_time_ms,
        "end_timestamp": end_time_ms,
        "resolution": resolution
    }
    r = requests.get(url, params=params)
    result = r.json().get("result", {})
    df = pd.DataFrame(result)
    if not df.empty:
        df['timestamp'] = pd.to_datetime(df['ticks'], unit='ms')
        df['instrument'] = instrument_name
    return df

# 3. 1년치 기간 설정 (시작 ~ 끝)
end_dt = datetime.utcnow()
start_dt = end_dt - timedelta(days=365)
end_ms = int(end_dt.timestamp() * 1000)
start_ms = int(start_dt.timestamp() * 1000)

# 4. 모든 옵션들에 대해 반복 수집
all_options = get_deribit_option_instruments()
all_data = []

In [6]:
len(all_options)

734

In [7]:
from tqdm import tqdm

for opt in tqdm(all_options):
    symbol = opt['instrument_name']
    try:
        df = get_option_candles(symbol, start_ms, end_ms, resolution='60')
        if not df.empty:
            all_data.append(df)
            # print(f"[OK] {symbol} - {len(df)} rows")
            time.sleep(0.5)  # API rate limit 보호용
        else:
            continue
            # print(f'[ERROR] Empty DF!: {symbol}')
    
    except Exception as e:
        print(f"[ERROR] {symbol}: {e}")

100%|██████████| 734/734 [23:10<00:00,  1.89s/it]


In [8]:
all_data[0]

,volume,ticks,status,open,low,high,cost,close,timestamp,instrument
0,0.6,1747645200000,ok,0.0002,0.0002,0.0003,0.00014,0.0003,2025-05-19 09:00:00,BTC-22MAY25-92000-P
1,0.3,1747648800000,ok,0.0004,0.0004,0.0004,0.00012,0.0004,2025-05-19 10:00:00,BTC-22MAY25-92000-P
2,0.0,1747652400000,ok,0.0004,0.0004,0.0004,0.00000,0.0004,2025-05-19 11:00:00,BTC-22MAY25-92000-P
3,0.0,1747656000000,ok,0.0004,0.0004,0.0004,0.00000,0.0004,2025-05-19 12:00:00,BTC-22MAY25-92000-P
4,2.6,1747659600000,ok,0.0003,0.0003,0.0003,0.00078,0.0003,2025-05-19 13:00:00,BTC-22MAY25-92000-P
5,0.2,1747663200000,ok,0.0003,0.0003,0.0003,0.00006,0.0003,2025-05-19 14:00:00,BTC-22MAY25-92000-P
6,0.0,1747666800000,ok,0.0003,0.0003,0.0003,0.00000,0.0003,2025-05-19 15:00:00,BTC-22MAY25-92000-P
7,0.0,1747670400000,ok,0.0003,0.0003,0.0003,0.00000,0.0003,2025-05-19 16:00:00,BTC-22MAY25-92000-P
8,0.0,1747674000000,ok,0.0003,0.0003,0.0003,0.00000,0.0003,2025-05-19 17:00:00,BTC-22MAY25-92000-P
9,1.0,1747677600000,ok,0.0002,0.0002,0.0002,0.00020,0.0002,2025-05-19 18:00:00,BTC-22MAY25-92000-P


In [9]:
final_df = pd.concat(all_data, ignore_index=True)

In [10]:
final_df.to_csv('btc_option.csv')

In [11]:
final_df

,volume,ticks,status,open,low,high,cost,close,timestamp,instrument
0,0.6,1747645200000,ok,0.0002,0.0002,0.0003,0.00014,0.0003,2025-05-19 09:00:00,BTC-22MAY25-92000-P
1,0.3,1747648800000,ok,0.0004,0.0004,0.0004,0.00012,0.0004,2025-05-19 10:00:00,BTC-22MAY25-92000-P
2,0.0,1747652400000,ok,0.0004,0.0004,0.0004,0.00000,0.0004,2025-05-19 11:00:00,BTC-22MAY25-92000-P
3,0.0,1747656000000,ok,0.0004,0.0004,0.0004,0.00000,0.0004,2025-05-19 12:00:00,BTC-22MAY25-92000-P
4,2.6,1747659600000,ok,0.0003,0.0003,0.0003,0.00078,0.0003,2025-05-19 13:00:00,BTC-22MAY25-92000-P
...,...,...,...,...,...,...,...,...,...,...
1097362,0.0,1747843200000,ok,0.0120,0.0120,0.0120,0.00000,0.0120,2025-05-21 16:00:00,BTC-27MAR26-320000-C
1097363,0.0,1747846800000,ok,0.0120,0.0120,0.0120,0.00000,0.0120,2025-05-21 17:00:00,BTC-27MAR26-320000-C
1097364,0.0,1747850400000,ok,0.0120,0.0120,0.0120,0.00000,0.0120,2025-05-21 18:00:00,BTC-27MAR26-320000-C
1097365,0.0,1747854000000,ok,0.0120,0.0120,0.0120,0.00000,0.0120,2025-05-21 19:00:00,BTC-27MAR26-320000-C


In [12]:
get_deribit_option_instruments()

[{'price_index': 'btc_usd',
  'rfq': False,
  'kind': 'option',
  'min_trade_amount': 0.1,
  'instrument_name': 'BTC-22MAY25-92000-C',
  'maker_commission': 0.0003,
  'taker_commission': 0.0003,
  'instrument_type': 'reversed',
  'expiration_timestamp': 1747900800000,
  'creation_timestamp': 1747641609000,
  'is_active': True,
  'tick_size': 0.0001,
  'contract_size': 1.0,
  'strike': 92000.0,
  'instrument_id': 465833,
  'settlement_period': 'day',
  'option_type': 'call',
  'block_trade_commission': 0.0003,
  'block_trade_min_trade_amount': 25,
  'block_trade_tick_size': 0.0001,
  'settlement_currency': 'BTC',
  'base_currency': 'BTC',
  'counter_currency': 'USD',
  'quote_currency': 'BTC',
  'tick_size_steps': [{'tick_size': 0.0005, 'above_price': 0.005}]},
 {'price_index': 'btc_usd',
  'rfq': False,
  'kind': 'option',
  'min_trade_amount': 0.1,
  'instrument_name': 'BTC-22MAY25-92000-P',
  'maker_commission': 0.0003,
  'taker_commission': 0.0003,
  'instrument_type': 'reversed',
 